<a href="https://colab.research.google.com/github/Amir5-Tech/ML-Object-Recognition/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook contains the driver code for constructing, loading the data and training of the image recognition model.
The model utilizes the pre-trained MobileNet model using fine tunning**

In [ ]:
 from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from matplotlib.pyplot import imread, imshow, title, pause

Before running this you have to upload the image_net.py and dataset.py files
Also it is possible to modify the categories that you wish to train on, provided that their images are in the correct path

In [ ]:
import dataset
import image_net
import cv2
cats = ['Can', 'Key', 'Tree', 'Cookie', 'Plate', 'None']

the dataset_fp is the root path which contains all of the images, provided that they're seperated by category

In [ ]:
dataset_fp = "drive/My Drive/Agora-Datasets/open_images"

the ImageDataset object allows me to interface with images in clean way

In [ ]:
my_dataset =dataset.ImageDataset(dataset_fp, cats, "july_24")

In [ ]:
import pandas as pd
from os.path import join
import tensorflow as tf


In [ ]:
my_dataset.save("./drive/My Drive/Agora-Datasets")

In [ ]:
from sklearn.model_selection import ShuffleSplit
from datetime import date
from numpy import reshape
# for cross-validation s
num_splits = 5
cv = ShuffleSplit(n_splits=num_splits, test_size=0.2, random_state=date.today().day)
batch_size = 32
image_paths, labels = my_dataset.format()


{0: 'Can', 1: 'Key', 2: 'Tree', 3: 'Cookie', 4: 'Plate', -1: 'None'}


In [ ]:
it = 0
from os.path import join
models_path = join("./", "drive", "My Drive", "Models")
for train_index_arr, test_index_arr in cv.split(image_paths, labels):
  g = tf.Graph()
  with tf.Session(graph=g) as sess:
    my_model = image_net.ImageRecModel(n_classes=6)
    train_dataset, val_dataset, test_dataset = my_dataset.as_tf_dataset(32, 
                                                                       train_index_arr, test_index_arr)
    my_model.train(train_dataset, val_dataset)
    my_model.save("./drive/My Drive/Models/model{}.js".format(it), 
                     "./drive/My Drive/Models/model{}.h5".format(it))
    it += 1
    del g


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                40992     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 102       
Total params: 2,299,606
Trainable params: 1,666,518
Non-trainable params: 633,088
_________________________________________________________________
None
compiled
done
{0: 'Can', 1: 'Key', 2: 'Tree', 3: 'Cookie', 4: 'Plate', -1: 'None'}
Tensor("args_1:0",

W0829 09:45:36.515438 140367499138944 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/rmsprop.py:119: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/50
65/65 [==============================] - 1111s 17s/step - loss: 0.7978 - acc: 0.4111 - val_loss: 0.7907 - val_acc: 0.4101
Epoch 2/50
65/65 [==============================] - 189s 3s/step - loss: 0.7439 - acc: 0.4675 - val_loss: 0.7375 - val_acc: 0.4754
Epoch 3/50
65/65 [==============================] - 185s 3s/step - loss: 0.6283 - acc: 0.6526 - val_loss: 0.6536 - val_acc: 0.6029
Epoch 4/50
65/65 [==============================] - 183s 3s/step - loss: 0.5413 - acc: 0.7651 - val_loss: 0.5849 - val_acc: 0.6986
Epoch 5/50
65/65 [==============================] - 181s 3s/step - loss: 0.4867 - acc: 0.8032 - val_loss: 0.5520 - val_acc: 0.7572
Epoch 6/50
65/65 [==============================] - 180s 3s/step - loss: 0.4482 - acc: 0.8378 - val_loss: 0.5051 - val_acc: 0.7949
Epoch 7/50
65/65 [==============================] - 181s 3s/step - loss: 0.4212 - acc: 0.8635 - val_loss: 0.4715 - val_acc: 0.8261
Epoch 8/50
65/65 [==============================] - 180s 3s/step - loss: 0.4016 -

In [ ]:
import importlib
importlib.reload(image_net)


importlib.reload(dataset)

<module 'dataset' from '/content/dataset.py'>

In [ ]:
print(my_dataset.name_to_label)
from os.path import join 
model_fp = join("./drive", "My Drive", "Models")
to_d = image_net.ImageRecModel().restore(join(model_fp, "model0.js"),
                                         join(model_fp, "model0.h5"))
to_d.model.save(join(model_fp, "to_d.h5"))


{'Can': 0, 'Key': 1, 'Tree': 2, 'Cookie': 3, 'Plate': 4}


KeyboardInterrupt: ignored

In [ ]:
  from os import listdir
from os.path import isdir
dataset = dataset.ImageDataset.load("//drive/My Drive/Agora-Datasets/july_18.json")

with tf.Session() as sess:
  image_path = tf.placeholder(dtype=tf.string)
  image = tf.io.read_file(image_path)
  image = tf.cond(tf.image.is_jpeg(image),
                  lambda: tf.image.decode_jpeg(image, channels=3,
                                               try_recover_truncated=True,acceptable_fraction=0.1),
                  lambda: tf.image.decode_png(image, channels=3))
  image = tf.cast(image, tf.float32)
  image = image / 127.5 - 1
  image = tf.image.resize(image, (224, 224))

  for cat in cats:
    ims = [join(dataset_fp, cat, x) for x in listdir(join(dataset_fp, cat)) if not isdir(join(dataset_fp, cat, x))]
    for im in ims:
      try:
         sess.run(image, feed_dict={
             image_path:im
         })
      
      except tf.errors.InvalidArgumentError:
        print(im)
    print(cat)

drive/My Drive/Agora-Datasets/open_images/Can/66.8edf6f17edee1e7ff1424216bc2a767f.jpeg
drive/My Drive/Agora-Datasets/open_images/Can/81.50pcs-fashion-12oz-17oz-Thermos-Double-Walled-Soda-Cans-Vacuum-Insulated-Stainless-Steel-Coffee-mugs-with.jpg
drive/My Drive/Agora-Datasets/open_images/Can/11.aluminum-red-soda-can-isolated-260nw-144526784.jpg
drive/My Drive/Agora-Datasets/open_images/Can/13.330-ml-aluminum-beverage-drink-260nw-551975491.jpg
drive/My Drive/Agora-Datasets/open_images/Can/77.open-aluminum-cans-isolated-on-260nw-1342901249.jpg
drive/My Drive/Agora-Datasets/open_images/Can/85.top-open-closed-1.gif
drive/My Drive/Agora-Datasets/open_images/Can/27.chanthaburithailandoctober-262018-coca-cola-can-260nw-1213013332.jpg
drive/My Drive/Agora-Datasets/open_images/Can/73.mykolaiv-ukraine-november-15-2018-260nw-1287411121.jpg
drive/My Drive/Agora-Datasets/open_images/Can/86.7ff253ffa24e46665413a4bff0490ef9.jpg
drive/My Drive/Agora-Datasets/open_images/Key/59.1561456401223776019_thumb

KeyboardInterrupt: ignored